In [ ]:
# runtime cpu
# 구글 드라이브 연동
# 경로 이동 -> 딥러닝 폴더까지

In [ ]:
from google.colab import drive
drive

In [1]:
%cd 'drive/MyDrive/Colab Notebooks/23.12.18딥러닝'

/content/drive/MyDrive/Colab Notebooks/23.12.18딥러닝


### 목표
- 개, 고양이 이미지(사진)을 배열로 변환하는 작업을 진행해보자(이미지 데이터 배열화)
  - 압축된 형식의 배열로 저장: npz
  - 대용량 데이터를 압축형채로 정리해두면 클라우그 환경에 쉽게 업로드, 다운로드 가능하잼 ( 관리가 편해짐)
  - 저장소의 공간을 적게 차지(공간 사용효율졸아짐)
  

In [ ]:
# 1. 압축 폴더 해제
# 2. 이미지 한장씩 접근
# 3. 이미지 크기 조정(통일)
# 4. 배열 변환 np.array(대상)
# 5. 모든 이미지를 npz 형식으로 저장
# * 답데이터 생성

In [3]:
# 1. 압축 폴더 해제
from zipfile import ZipFile
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image #!pip install PIL

In [5]:
zip_file_path='./data/cats_and_dogs_filtered.zip'
# 파일 다루는 코드
with ZipFile(zip_file_path,'r') as f:
  # path 압축 파일 해제 후 폴더 저장할 경로 설정
  f.extractall(path='./data/')

In [7]:
# 데이터 폴더 > 압축 해제된 폴더 있는지 확인
# train- cats, dogs
# test- cats, dogs
# 총 4개의 폴더 경로를 변수에 저장
train_cats_dir='./data/cats_and_dogs_filtered/train/cats/'
train_dogs_dir='./data/cats_and_dogs_filtered/train/dogs/'
test_cats_dir='./data/cats_and_dogs_filtered/test/cats/'
test_dogs_dir='./data/cats_and_dogs_filtered/test/dogs/'

# 각 폴더에 있는 이미지 파일명 접근
# os.listdir(폴더경로) : 해당 경로에 있는 파일명을 리스트로 출력
train_cats_fn=os.listdir(train_cats_dir)
train_dogs_fn=os.listdir(train_dogs_dir)
test_cats_fn=os.listdir(test_cats_dir)
test_dogs_fn=os.listdir(test_dogs_dir)


In [8]:
# 각각의 이미지 파일이 몇개씩 들어있는지 개수 확인
print(len(train_cats_fn),len(train_dogs_fn))
print(len(test_cats_fn),len(test_dogs_fn))


1000 1000
500 500


In [16]:
# 예시
# 이미지 1장 크기 조정 후 arr 변환
# train_cats_fn[0] 진행
tmp_path=os.path.join(train_cats_dir,train_cats_fn[0])
tmp_path
# train_cats_dir + '/'+ train_cats_fn[0]

img=Image.open(tmp_path).resize((224,224)) # 정사각형
type(np.array(img))

numpy.ndarray

In [ ]:
# 전처리하는 함수 정의 (생성)
# 사진을 불러와서 배열로 바꾸는 함수
# 매개변수: 폴더명, 각 파일명 변수, 이미지 사이즈=(224,224)

In [22]:
def prepro_imgs(folder_path,file_names,img_size=(224,224)):
  # 이미지들을 묶어줄 빈 리스트 선언
  imgs=[] # 1장씩 arr변환될때 append 이미지 누적 저장

  # 각 이미지 접근해서 불러오는 작업 > 크기 조정 > arr > imgs 저장
  for i in file_names :
    # 폴더 경로 + 이미지 파일명 합치기
    path=os.path.join(folder_path,i)
    # 해당 경로 이미지 불러 -> 크기 조정
    img=Image.open(path).resize(img_size)
    # img -> array -> imgs  리스트 누적 저장
    imgs.append(np.array(img))

  return np.array(imgs) # 모델 연결 array 사용

In [24]:
# train-cats,dogs
# test-cats,dogs
X_train_cats=prepro_imgs(train_cats_dir,train_cats_fn)
X_train_dogs=prepro_imgs(train_dogs_dir,train_dogs_fn)
X_test_cats=prepro_imgs(test_cats_dir,test_cats_fn)
X_test_dogs=prepro_imgs(test_dogs_dir,test_dogs_fn)
X_train_cats.shape

(1000, 224, 224, 3)

In [25]:
# 크기 확인
print('훈련셋',X_train_cats.shape,X_train_dogs.shape)
print('훈련셋',X_test_cats.shape,X_test_dogs.shape)

# 1000장 고양이 사진, 행 224, 열 224, 채널의 수 rgb(3)
# 1장 행 224, 열 224 채널 수 3 (컬러 이미지)

훈련셋 (1000, 224, 224, 3) (1000, 224, 224, 3)
훈련셋 (500, 224, 224, 3) (500, 224, 224, 3)


In [26]:
# train cats, dogs 병합
X_train=np.concatenate((X_train_cats,X_train_dogs))
# test cats, dogs 병합
X_test=np.concatenate((X_test_cats,X_test_dogs))

# 크기 확인
print(X_train.shape)
print(X_test.shape)


(2000, 224, 224, 3)
(1000, 224, 224, 3)


In [27]:
# 답데이터 생성하기
# numpy 1차원 배열 생성
# cat 0, dog 1 답데이터 정리
y_train=np.array([0]*1000+[1]*1000)
y_test=np.array([0]*500+[1]*500)
print(y_train.shape,y_test.shape)

(2000,) (1000,)


In [28]:
# 데이터 셋팅 완료 > npz
# X_train, X_test,y_train, y_test >npz
np.savez_compressed('./data/np_cats_vs_dogs.npz',
                    X_train=X_train,
                    X_test=X_test,
                    y_train=y_train,
                    y_test=y_test)

In [ ]:
# 해당 경로에 npz 데이터 생성 됐는지 확인 하기